In [ ]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from collections import OrderedDict
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import folium
from datetime import date
import argparse
from glob import iglob
import os
from pathlib import Path
import pickle
import harp
import numpy as np
import xarray as xr
import rioxarray as rio 
from glob import glob
import os

from matplotlib import pyplot
from rasterio.plot import show
from rasterio.plot import show_hist
import matplotlib.pyplot as plt

import numpy as np

import rasterio
from rasterio.warp import calculate_default_transform, reproject
from rasterio.enums import Resampling

from rasterio.windows import Window

### 1. Download Sentinel5P data from SentinelSat API

Searching Sentinel5p data works best by using a geojson file of your area of interest. For this project, the coordinates of the GHGSat target data is used. Function can be used without the footprint.

CH4 products older than 1 year are offline and archived. By making this API call, it will trigger the request to make them available.

In [ ]:
def get_products(api, product, level, date_range, footprint=None):
        """ Search the API based on polygon and time period. Return the products and metadata."""
        if footprint is not None:
            products = api.query(footprint,
                            date=(date_range[0], date_range[1]), 
                            area_relation='Intersects',
                            platformname='Sentinel-5',
                            producttype=product,
                            processinglevel=level,
                            processingmode='Offline'
                            )
        else: 
            products = api.query(date=(date_range[0], date_range[1]), 
                            platformname='Sentinel-5',
                            producttype=product,
                            processinglevel=level,
                            processingmode='Offline'
                            )
        products_df = api.to_dataframe(products) 

        return products, products_df

In [ ]:
path ="/Users/amycairns/Desktop/example_aoi.geojson"
footprint = geojson_to_wkt(read_geojson(path))

api = SentinelAPI('s5pguest', 's5pguest', api_url='https://s5phub.copernicus.eu/dhus/') # all users should use these details, not sensitive

products, products_df = get_products(api, 
                                     product='L2__CH4___', 
                                     footprint= footprint, 
                                     level='L2', 
                                     date_range=('20220925', '20220926'))

print(len(products))

In [ ]:
api.download_all(products)

## 2. Crop data to area of interest


In [ ]:
def get_location_bbx(location):
    """ Retrieve coordinates for the location: minimum latitude, maximim latitude, mininimum longitude and maximum longitude """ 
    print("Getting coordinates for:", location)
    if location == 'California':
         location_bbox = [34.9, 35.1, -118.5, -119] #example location
    else:
        raise Exception('location {location} is not defined'.format(location))
        
    coords_dict = {}
    coords_dict['min_lat'] = location_bbox[0]
    coords_dict['max_lat'] = location_bbox[1]
    coords_dict['min_lon'] = location_bbox[2]
    coords_dict['max_lon'] = location_bbox[3]

    return coords_dict

In [ ]:
fields = 'latitude_bounds, longitude_bounds, latitude, longitude, '\
             + 'cloud_fraction, sensor_altitude, sensor_azimuth_angle, sensor_zenith_angle, '\
             + 'solar_azimuth_angle, solar_zenith_angle, '\
             + 'CH4_column_volume_mixing_ratio_dry_air_uncertainty'

def aoi_steps(location, degrees):
    """ calculate the steps and degrees for the aoi """
    location_coords = get_location_bbx(location)

    # compute the steps
    lat_steps = (location_coords['max_lat'] - location_coords['min_lat'])/degrees
    lon_steps = (location_coords['max_lon'] - location_coords['min_lon'])/degrees
    lat_steps, lon_steps = int(np.ceil(lat_steps)), int(np.ceil(lon_steps))

    print(location_coords['max_lat'], location_coords['min_lat'] + degrees*lat_steps)
    print(location_coords['max_lon'], location_coords['min_lon'] + degrees*lon_steps)

    return lat_steps, lon_steps, location_coords


def retrieve_files(folder_raw):
    """ search for .nc files in the location """
    path_files = os.path.join(folder_raw, '*.nc')
    all_files = sorted(list(iglob(path_files, recursive=True)))

    print("looking for files at: ", (path_files))
    print("number of .nc detected: ", len(all_files))

    return all_files

def get_harp_operations(location,
                        degrees = 0.01, 
                        product = 'L2__CH4___', 
                        harp_product= 'CH4_column_volume_mixing_ratio_dry_air', 
                        threshold = 75, 
                        keep_general=fields):
    """ set up the operations that harp needs to create the grid """

    # get aoi params
    lat_steps, lon_steps, location_coords = aoi_steps(location, degrees)

    # define harp operations
    ops_string = f"{harp_product}_validity > {threshold}; \
        derive(datetime_stop {{time}});\
        latitude >= {location_coords['min_lat']-1} [degree_north] ; latitude <= {location_coords['max_lat']+1} [degree_north] ;\
        longitude >= {location_coords['min_lon']-1} [degree_east] ; longitude <= {location_coords['max_lon']+1} [degree_east];\
        bin_spatial({lat_steps+1}, {location_coords['min_lat']}, {degrees}, {lon_steps+1}, {location_coords['min_lon']}, {degrees});\
        derive(latitude {{latitude}}); derive(longitude {{longitude}});\
        keep({harp_product}, {keep_general})"

    print('operations:\n', ops_string, '\n')

    return ops_string


In [ ]:
def process(location, folder_raw, folder_output):
    """ Run the processing and save successful .nc files. """
    success_files, unsuccessful_files = [], []
    print(location)
    
    ## get files to be processed
    all_files = retrieve_files(folder_raw)
    print("retrieved files")
    
    ## get harp operations
    ops_string = get_harp_operations(location)

    for i, file in enumerate(all_files):
        try:
            harp_L2_L3 = harp.import_product(file, operations=ops_string, options= 'ch4=bias_corrected')
            export_path = '{}{}.{}'.format(folder_output, file.split("/")[-1].replace('L2', 'L3').split('.')[0], 'nc')
            print("export_path: ", export_path)
            harp.export_product(harp_L2_L3, export_path, file_format='netcdf')
            success_files.append(file)
        except:
            unsuccessful_files.append(file)
    
    print("The following files have been successful:\n", success_files)
    print("The cropped data will be stored in:"+folder_output)
    print("Warning - The following files have no data in your aoi\n", unsuccessful_files)

In [ ]:
process(location='California',
        folder_output='/Users/amycairns/Desktop/',
        folder_raw='/Users/amycairns/Desktop/Dissertation_data/GHGSat-observations/Bakersfield/observation-AXuDSFy-769909-arm207@bath.ac.uk-2023-03-15/Sentinel-5p/S5P_OFFL_L2__CH4____20220213T191954_20220213T210124_22479_02_020301_20220215T112215.nc')


## 3. Resample data

In [ ]:
def change_crs(in_path, out_path, new_crs):
    """ Reproject files and update metadata """
    with rasterio.open(in_path) as source:
        source_crs = source.crs
        print(source_crs)
        transform, width, height = calculate_default_transform(source_crs, new_crs, source.width, source.height, *source.bounds)
        kwargs = source.meta.copy()

        kwargs.update({
            "driver": 'GTiff',
            'crs': new_crs,
            'transform': transform,
            'width': width,
            'height': height})
        print(source.meta)
        
        with rasterio.open(out_path, 'w', **kwargs) as dst:
            print(out_path)
            for i in range(1, source.count + 1):
                print(source.count)
                reproject(
                    source=rasterio.band(source, i),
                    destination=rasterio.band(dst, i),
                    src_transform=source.transform,
                    src_crs=source.crs,
                    dst_transform=transform,
                    dst_crs=new_crs,
                    resampling=Resampling.nearest)
                print(source.crs)
    return(out_path)

In [ ]:
base_path = '/Users/amycairns/Desktop/Dissertation_data/GHGSat-observations/'
locations = ['California', 'Arizona']

# folder structure for this loop is: location (Arizona or California) > Observation id
for location in locations:
    for observation in os.listdir(os.path.join(base_path, location)):
        f_name = os.path.splitext(os.path.basename(f))[0]
        nc_file = xr.open_dataset(f)
        
        #select field of interest
        ch4 = nc_file['CH4_column_volume_mixing_ratio_dry_air']

        #set longitude and latitude dimensions
        ch4 = ch4.rio.set_spatial_dims(x_dim='longitude', y_dim='latitude')
        ch4.rio.write_crs("epsg:4326", inplace=True) # needs to know the previous crs
        out_path = os.path.join(base_path, location, observation, "Sentinel-5p/crop/", f_name+".tiff")
        ch4.rio.to_raster(os.path.join(Sentinel_5_path, out_path))

        #Define the CRS projection
        if location == "California":
            crs = "EPSG:32611"
            change_crs(out_path, out_path, crs)

        elif location == "Arizona":
            crs = "EPSG:32612"
            change_crs(out_path, out_path, crs)

        else: 
            raise Exception('location is not defined')

In [ ]:
def resample(path, out_dir, target_resolution): 
    
    with rasterio.open(path) as dataset:
        resampling_factor = dataset.transform[0]/target_resolution
        if resampling_factor != 1:
            # resample data to target pixel size
            data = dataset.read(
                out_shape=(
                    dataset.count,
                    int(dataset.height * resampling_factor),
                    int(dataset.width * resampling_factor)
                ),
                resampling=Resampling.bilinear
            )

            # image transform
            transform = dataset.transform * dataset.transform.scale(
                (dataset.width / data.shape[-1]),
                (dataset.height / data.shape[-2])
            )
            out_meta = dataset.meta.copy()
            out_height = int(dataset.height * resampling_factor)
            out_width = int(dataset.width * resampling_factor)
            crs = dataset.crs
            out_meta.update({"driver":"GTiff",
                            "height": out_height,
                            "width": out_width,
                            "transform": transform,
                            "crs" : crs})

            out_path = os.path.join(out_dir, "resample_"+str(resampling_factor)+"_"+os.path.basename(path))
            with rasterio.open(out_path,"w",**out_meta) as dest:
                dest.write(data)

        else:  
            shutil.copy(path, out_dir)

In [ ]:
#iterate all sentinel 5 crop folders for each observation and resample
for location in locations:
    for observation in os.listdir(os.path.join(base_path, location)):
            for file in glob(os.path.join(base_path, location, observation, "Sentinel-5p/crop/", "*.tiff")):
                print(file)
                out_dir = os.path.join(base_path, location, observation, "Sentinel-5p/resample")
                resample(file, out_dir, 10)

In [ ]:
#test
example  = "file"

with rasterio.open(example) as s5p:
    print(s5p.count) 
    data_array = np.array(s5p.read())
print(s5p.bounds)
print(s5p.crs)
print(np.shape(data_array))

plt.imshow(data_array[0])
plt.show()

## 3. Crop to 15x20km

Unable to use the previous approach since now the data is in tiff files.

In [ ]:
def create_window(mid_height, mid_width, window_height, window_width):
    min_height, max_height =  mid_height - (window_height/2), mid_height + (window_height/2) 
    min_width, max_width = mid_width - (window_width/2), mid_width + (window_width/2)
    window = Window.from_slices((int(min_height), int(max_height)),(int(min_width), int(max_width)))
    return window

for location in locations:
    for observation in os.listdir(os.path.join(base_path, location)):
            for file in glob(os.path.join(base_path, location, observation, "Sentinel-5p/resample/resample_*.tiff")):
                with rasterio.open(file) as src:
                    # see before
                    print(file)
                    print("width:", src.width)
                    print("height", src.height)
                    data_array = np.array(src.read())
                    plt.imshow(data_array[0])
                    plt.show()

                    #set variables
                    window_width, window_height = 1500, 2000 # desired pixels in window
                    mid_width, mid_height = src.width/2, src.height/2  

                    # create and apply the window, and show
                    window = create_window(mid_height, mid_width, window_height, window_width)
                    arr = src.read(1, window=window)
                    print("height, width:", np.shape(arr))
                    plt.imshow(arr)
                    plt.show()

                    #update metadata
                    meta = src.meta.copy()
                    crs = src.crs
                    meta.update({"driver":"GTiff",
                                    "height": window_height,
                                    "width": window_width,
                                    "transform": src.window_transform(window),
                                    "crs" : crs})
                    print(meta)

                    #save windowed version
                    with rasterio.open(os.path.join(base_path, location, observation, 'Sentinel-5p/crop_resample.tif'), 'w', **meta) as dst:
                        dst.write(src.read(window=window))

In [ ]:
#test
example_path = 'example_path'

with rasterio.open(os.path.join(example_path)) as s5p:
    data_array = np.array(s5p.read())
    print("aoi coordinates: ", s5p.bounds)
    print("crs: ", s5p.crs)
    print("shape: ", data_array.shape)
    print("maximum methane value :", data_array.max())

    fig, (axrgb, axhist) = pyplot.subplots(1, 2, figsize=(14,7))
    show(data_array[0], ax=axrgb)
    show_hist(data_array[0], bins=50, histtype='stepfilled',lw=0.0, stacked=False, alpha=0.3, ax=axhist)
    pyplot.show()

## 4. Replacing NAs with Gaussian distribution around mean

In [ ]:
for location in locations:
    for observation in os.listdir(os.path.join(base_path, location)):
        for file in glob(os.path.join(base_path, location, observation, "Sentinel-5p/crop_resample.tif")):
            with rasterio.open(file) as src:
                print(file)
                data_array = np.array(src.read()[0])
                data_array = np.where(data_array == 0.0, np.nan, data_array)
                plt.imshow(data_array)
                plt.show()
                print(data_array)
                print(np.shape(data_array))
                median = np.nanmedian(data_array)
                sd = np.nanstd(data_array)

                # using a gaussian as the data is a gaussian distribution
                nans = np.where(np.isnan(data_array)) #positions of the nans
                nan_length = len(data_array[nans])

                rand = np.random.normal(median, sd, nan_length)
                for i in range(nan_length):
                    data_array[nans[0][i]][nans[1][i]] = rand[i]
                plt.imshow(data_array)
                plt.show()
                np.save(os.path.join(base_path, location, observation, "Sentinel-5p/low_res_methane.npy"), data_array)